In [ ]:
import os 
import pandas as pd

input_folder = "/mnt/samsung_2tb/mixed_data/output/pca_transformed_results/output/"

# df = pd.read_parquet(os.path.join(input_folder, "output_dataframe_0.parquet"))



In [ ]:
da = pd.DataFrame()

for i in range(4):
    path = os.path.join(input_folder, f"output_dataframe_{i}.parquet")

    da= da.append(pd.read_parquet(path))


In [ ]:
da.count

In [ ]:
df["PCA_1"].to_numpy()

In [ ]:
from tdigest import TDigest
import numpy as np

digest = TDigest()

digest.batch_update(da["PCA_1"].to_numpy())


In [ ]:
digest.percentile(10)

-9.62286448783357

In [ ]:
import joblib 
import pickle

with open("tdigest.pkl", 'rb') as f:
    digest = pickle.load(f)

In [5]:
import dask.array as da

# Create a large Dask array (simulate distributed data)
data = da.random.random(size=1_100_000_000, chunks=100_000)  # 1M elements, 10 chunks

# Percentiles to compute
percentiles = [10, 25, 50, 75, 90]

# Calculate percentiles using TDigest method
result = da.percentile(data, q=percentiles, method='linear', internal_method='tdigest')

# Compute and print the results
print("Percentiles:", result.compute())


Percentiles: [0.10017561 0.25009912 0.49998359 0.74995481 0.89985113]


In [12]:
import dask.dataframe as dd
import dask.array as da
import numpy as np
# Define the input path
input_folder = "/mnt/samsung_2tb/mixed_data/output/pca_transformed_results/output/*.parquet"

# Read the Parquet files, selecting only 'PCA_1' column
df = dd.read_parquet(input_folder, columns=['PCA_1', 'PCA_2', 'PCA_3'], engine='pyarrow')  # or engine='pyarrow'


# Convert 'PCA_1' column to a Dask array
pca1_array = df['PCA_1'].to_dask_array(lengths=True)

# Define the percentiles to compute
percentiles_to_compute = [i for i in np.arange(100, step=2)]

# Compute the percentiles using Dask's internal method
percentiles = da.percentile(
    pca1_array, 
    q=percentiles_to_compute, 
    method='linear', 
    internal_method='tdigest'  # or 'tdigest' for memory efficiency
).compute()

# Print the results
for q, value in zip(percentiles_to_compute, percentiles):
    print(f"{q}th percentile of PCA_1: {value}")


0th percentile of PCA_1: -22.3494601290862
2th percentile of PCA_1: -13.146549090536524
4th percentile of PCA_1: -12.151756258227447
6th percentile of PCA_1: -11.151671236329406
8th percentile of PCA_1: -9.592409584386983
10th percentile of PCA_1: -8.254401694966019
12th percentile of PCA_1: -7.484747681025569
14th percentile of PCA_1: -6.911097772465119
16th percentile of PCA_1: -6.437421390527987
18th percentile of PCA_1: -6.040093269518786
20th percentile of PCA_1: -5.687396672472152
22th percentile of PCA_1: -5.357547216571537
24th percentile of PCA_1: -5.030769196497266
26th percentile of PCA_1: -4.695513839153042
28th percentile of PCA_1: -4.35916202040532
30th percentile of PCA_1: -4.004679122419613
32th percentile of PCA_1: -3.6233083642575696
34th percentile of PCA_1: -3.206035997114414
36th percentile of PCA_1: -2.751876987810707
38th percentile of PCA_1: -2.2742940458320082
40th percentile of PCA_1: -1.818145589954938
42th percentile of PCA_1: -1.3608777530485376
44th percen

In [13]:
df = df[df["PCA_1"] < -13.146549090536524]

In [15]:
df.count().compute()

PCA_1    134206902
PCA_2    134206902
PCA_3    134206902
dtype: int64